In [2]:
using DataFrames

In [3]:
df = readtable("data.csv");

## DataFrames

### DataFrame Methods

There are various simple methods you can use to inspect a `DataFrame`

In [4]:
size(df)

(9998,39)

In [5]:
names(df)

39-element Array{Symbol,1}:
 :timestamp           
 :page_group          
 :geo_cc              
 :geo_rg              
 :geo_city            
 :geo_org             
 :geo_netspeed        
 :user_agent_family   
 :user_agent_major    
 :user_agent_os       
 :user_agent_osversion
 :user_agent_model    
 :referrer            
 ⋮                    
 :dt_year             
 :dt_month            
 :dt_week             
 :dt_day              
 :dt_hour             
 :dt_6hour            
 :dt_12hour           
 :dt_minute           
 :dt_5min             
 :dt_15min            
 :dt_20min            
 :dt_30min            

### Each column of a DataFrame is a DataArray

You can reference a column using the column name as a `Symbol` subscript.  A `DataArray` is just a regular array that can contain `NA`, which is Juliaspeak for `NULL`.

In [6]:
df[:timers_t_done]

9998-element DataArray{Int64,1}:
  5568
  7853
  2662
  1605
  6674
  2289
 11595
  5019
 31749
  9041
  4844
 14059
  7482
     ⋮
   732
  3383
  3070
 26517
  9474
  6867
  4547
   530
  1655
  7021
  4693
  1790

In [7]:
df[30:40, :timers_t_done]

11-element DataArray{Int64,1}:
 44053
  2164
  1845
  5315
  3701
  7582
  2754
   973
  2914
 11527
  7947

In [11]:
df[30:40, [:timestamp, :geo_cc, :geo_netspeed, :user_agent_family, :timers_t_done]]

,timestamp,geo_cc,geo_netspeed,user_agent_family,timers_t_done
1,1409608800725,US,Cellular,Mobile Safari,44053
2,1409608800738,US,Cable/DSL,Firefox,2164
3,1409608800767,US,Cable/DSL,Chrome,1845
4,1409608800771,US,Cable/DSL,Chrome,5315
5,1409608800782,US,Cable/DSL,Chrome,3701
6,1409608800814,US,Cable/DSL,IE,7582
7,1409608800817,US,Cable/DSL,Chrome,2754
8,1409608800872,US,Cable/DSL,IE,973
9,1409608800930,US,Cable/DSL,Safari,2914
10,1409608800956,US,Cable/DSL,IE,11527


## Stats on DataFrames

Most Julia stats functions run on `AbstractArray`, which is the base type for `Array` as well as `DataArray`, so you can run them on any column of a `DataFrame` that contains numbers. You will probably need to remove `NA`s first using the `dropna` function.

Our test dataset doesn't contain any `NA` values for the `timers_t_done` column, so we're safe.

In [7]:
summarystats(df[:timers_t_done])

Summary Stats:
Mean:         6190.443989
Minimum:      15.000000
1st Quartile: 3078.250000
Median:       4449.500000
3rd Quartile: 6639.250000
Maximum:      551767.000000


## Histograms

The `hist` function will by default split the dataset into equal sized buckets based on the data's range.  This may not always be what you want, so you can pass in a list of thresholds as the second parameter.

The `hist` function returns a tuple.  The first element is the thresholds used, which might be a `Range` object or an `Array`.  The second element is the list of bucket frequencies.

In [8]:
hist(df[:timers_t_done])

(0.0:50000.0:600000.0,[9954,32,5,0,2,3,1,0,0,0,0,1])

### Creating thresholds based on the data

We could use static thresholds, but that wouldn't adapt to different data sets.  In this case, we develop a Julia function that determines thresholds based on the dataset.

Rather than divide the entire range into a fixed set of buckets, we divide the Inter-Quartile Range.  This has the advantage of excluding outliers from the basic range.  We then include outliers in their own buckets, one for the low bound and one for the high bound.

This is very similar to a box and whiskers plot.

In [17]:
# Function to set histogram thresholds after dropping outliers based on IQR
function getSymmetricThresholds(results::DataFrame; timer::Symbol=:timers_t_done)
    summary = summarystats(results[timer])
    fw  = (summary.q75-summary.q25)*1.5

    low = round(Int64, max(summary.min, summary.q25-fw))
    high = round(Int64, min(summary.max, summary.q75+fw))+1

    thresholds::Array{Int64, 1} = []

    nthresholds=25

    range = high - low

    for i in 0:nthresholds-1
        push!(thresholds, round(Int64, low + i * range/nthresholds))
    end

    push!(thresholds, high)
    if high < round(Int64, summary.max)
        push!(thresholds, round(Int64, summary.max))
    end

    return thresholds
end

getSymmetricThresholds (generic function with 1 method)

#### Julia Functions

Notice that Julia functions are declared using the `function` keyword.  Function parameters may have types attached to them, this is optional, and mainly useful when you use overload function names.

Functions may have optional parameters, a `;` separates required parameters from optional ones.

When passing optional parameters to a function, they need to be passed by name, and order doesn't matter.

A function typically only returns a single value, though that value may be a tuple of multiple objects.  The caller can then receive the return value into a single tuple or multiple values enclosed in `()`.

In [18]:
thresholds = getSymmetricThresholds(df)

27-element Array{Int64,1}:
     15
    494
    972
   1451
   1930
   2408
   2887
   3366
   3844
   4323
   4802
   5280
   5759
      ⋮
   7195
   7674
   8153
   8631
   9110
   9589
  10067
  10546
  11025
  11503
  11982
 551767

Running the `hist` function using our new thresholds gets us much better granularity into the data.

In [11]:
hist_global = hist(df[:timers_t_done], thresholds)[2]

26-element Array{Int64,1}:
  36
  90
 168
 370
 589
 867
 951
 873
 838
 752
 637
 542
 479
 359
 303
 291
 199
 186
 146
 119
 115
 100
  77
  90
  67
 753

## Filtering DataFrames

We can also filter a `DataFrame` on the value of one or more fields.  In the following example, we filter on all `:geo_rg` that are not `NA` and equal to `US:: OR`.

In [ ]:
results_OR = df[!isna(df[:geo_rg]) & (df[:geo_rg] .== "US:: OR"), :];

In [13]:
hist_OR = hist(results_OR[:timers_t_done], thresholds)[2]

26-element Array{Int64,1}:
  2
  0
  0
  2
  4
  6
  6
  5
  6
  3
  3
  7
  0
  7
  6
  3
  3
  1
  0
  1
  2
  4
  1
  1
  2
 14

### Statistical Correlation

The `cor` function lets us run a correlation between the two histograms that we have

In [14]:
cor(hist_global, hist_OR)

0.6190857138462522

We could also run `cumsum` to generate the `CDF` from the histogram and correlate those values.

In [15]:
cor(cumsum(hist_global), cumsum(hist_OR))

0.9836407166342165

## Splitting/Grouping a DataFrame

Use the `by` function to run an aggregation on a DataFrame grouped by one or more columns

In [23]:
by(df, :user_agent_family, rows -> median(rows[:timers_t_done]))

,user_agent_family,x1
1,AOL,7119.0
2,Android Browser,14062.0
3,BlackBerry WebKit,11658.0
4,Chrome,3961.0
5,Chrome Mobile,8059.0
6,Chrome Mobile iOS,8589.0
7,Firefox,4978.0
8,IE,5047.0
9,IE Mobile,14156.0
10,Iceweasel,5657.0


### Problems if the aggregation function returns an array

If the aggregation function returns an array, like the `hist` function does, then we'll actually end up with one row per array element.  Instead we need to serialize the array to a string or create a custom data type that encapsulates the array.  The string method is easier albeit a little slower, but if we're going to export our data to JavaScript, we may need to do this anyway.

In [28]:
by(
    df,
    :user_agent_family, 
    rows -> DataFrame(
        count = size(rows, 1),
        median = median(rows[:timers_t_done]),
        hist = JSON.json(hist(rows[:timers_t_done], thresholds)[2])
    )
)

,user_agent_family,count,median,hist
1,AOL,5,7119.0,"[0,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0,0,1,1,0,0,0,0,0,0,0]"
2,Android Browser,32,14062.0,"[0,0,0,0,1,0,0,0,1,1,1,2,0,0,1,0,0,2,1,0,1,1,0,0,0,20]"
3,BlackBerry WebKit,3,11658.0,"[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,1]"
4,Chrome,3423,3961.0,"[15,39,81,174,283,367,371,311,309,255,208,161,135,100,89,75,69,52,27,27,28,16,20,21,12,178]"
5,Chrome Mobile,57,8059.0,"[1,0,0,0,0,0,0,1,2,2,1,4,2,0,6,8,2,1,5,1,1,0,3,2,3,12]"
6,Chrome Mobile iOS,3,8589.0,"[0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1]"
7,Firefox,1609,4978.0,"[5,8,17,32,62,95,112,148,153,121,122,93,75,58,61,66,40,33,38,28,21,29,8,19,10,155]"
8,IE,3309,5047.0,"[6,25,34,75,136,234,270,253,243,268,226,214,204,152,122,110,69,68,50,46,54,48,37,37,36,291]"
9,IE Mobile,20,14156.0,"[0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,2,0,2,2,1,12]"
10,Iceweasel,1,5657.0,"[0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0]"


### Copy the JSON to a JavaScript file when first testing D3 code

It's easier to start your D3 experimentation with a standalone file rather than within the IJulia interface.  A simpler dev setup is easier to debug.

In [21]:
println("Histogram:\n", JSON.json(hist_global))
println()
println("Thresholds:\n", JSON.json(thresholds))

Histogram:
[36,90,168,370,589,867,951,873,838,752,637,542,479,359,303,291,199,186,146,119,115,100,77,90,67,753]

Thresholds:
[15,494,972,1451,1930,2408,2887,3366,3844,4323,4802,5280,5759,6238,6717,7195,7674,8153,8631,9110,9589,10067,10546,11025,11503,11982,551767]
